<a href="https://colab.research.google.com/github/binliu0630/Data_Manipulation/blob/master/Minimally_Sufficient_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Inspired by https://colab.research.google.com/drive/1_9fwgpRrQupG1uBunaS2-ySfr0dXdfcb#scrollTo=DvKPqCGSJpuz

- Selecting a single column of data
- The deprecated ix indexer
- Selection with at and iat
read_csv vs read_table duplication
isna vs isnull and notna vs notnull
Arithmetic and Comparison Operators and their Corresponding Methods
Builtin Python functions vs Pandas methods with the same name
Standardizing groupby aggregation
Handling a MultiIndex
The similarity between groupby, pivot_table and crosstab
pivot vs pivot_table
The similarity between melt and stack
The similarity between pivot and unstack


**Read in data**
- Only use read_csv to read in delimitted text files

In [0]:
import pandas as pd

data = pd.read_csv('https://ed-public-download.app.cloud.gov/downloads/Most-Recent-Cohorts-Scorecard-Elements.csv') # instead of pd.read_table('', delimiter=', ')

In [3]:
data.head(2)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,INSTURL,NPCURL,HCM2,PREDDEG,HIGHDEG,CONTROL,LOCALE,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,RELAFFIL,SATVR25,SATVR75,SATMT25,SATMT75,SATWR25,SATWR75,SATVRMID,SATMTMID,SATWRMID,ACTCM25,ACTCM75,ACTEN25,ACTEN75,ACTMT25,ACTMT75,ACTWR25,ACTWR75,...,CIPTITLE5,CIPTITLE6,CIPTFBS1,CIPTFBS2,CIPTFBS3,CIPTFBS4,CIPTFBS5,CIPTFBS6,CIPTFBSANNUAL1,CIPTFBSANNUAL2,CIPTFBSANNUAL3,CIPTFBSANNUAL4,CIPTFBSANNUAL5,CIPTFBSANNUAL6,MTHCMP1,MTHCMP2,MTHCMP3,MTHCMP4,MTHCMP5,MTHCMP6,OMENRYP_ALL_POOLED_SUPP,OMENRAP_ALL_POOLED_SUPP,OMAWDP8_ALL_POOLED_SUPP,OMENRUP_ALL_POOLED_SUPP,OMENRYP_FIRSTTIME_POOLED_SUPP,OMENRAP_FIRSTTIME_POOLED_SUPP,OMAWDP8_FIRSTTIME_POOLED_SUPP,OMENRUP_FIRSTTIME_POOLED_SUPP,OMENRYP_NOTFIRSTTIME_POOLED_SUPP,OMENRAP_NOTFIRSTTIME_POOLED_SUPP,OMAWDP8_NOTFIRSTTIME_POOLED_SUPP,OMENRUP_NOTFIRSTTIME_POOLED_SUPP,OMENRYP_FULLTIME_POOLED_SUPP,OMENRAP_FULLTIME_POOLED_SUPP,OMAWDP8_FULLTIME_POOLED_SUPP,OMENRUP_FULLTIME_POOLED_SUPP,OMENRYP_PARTTIME_POOLED_SUPP,OMENRAP_PARTTIME_POOLED_SUPP,OMAWDP8_PARTTIME_POOLED_SUPP,OMENRUP_PARTTIME_POOLED_SUPP
0,100654,100200,1002,Alabama A & M University,Normal,AL,www.aamu.edu/,https://galileo.aamu.edu/NetPriceCalculator/np...,0,3,4,1,12.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,365.0,485.0,360.0,495.0,370.0,457.0,425.0,428.0,414.0,16.0,19.0,14.0,20.0,15.0,18.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0152,0.3255,0.3645,0.2947,0.0163,0.3273,0.3532,0.3032,0.0113,0.3194,0.4048,0.2645,0.0137,0.315,0.3755,0.2958,0.0289,0.4224,0.2635,0.2852
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,www.uab.edu,uab.studentaidcalculator.com/survey.aspx,0,3,4,1,12.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,440.0,630.0,550.0,740.0,NaN,NaN,535.0,645.0,NaN,21.0,28.0,22.0,31.0,21.0,26.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0828,0.2588,0.5591,0.0993,0.0786,0.2684,0.5716,0.0814,0.0876,0.2477,0.5448,0.12,0.0728,0.2583,0.5872,0.0817,0.1398,0.2618,0.3977,0.2008


**Selecting data**
- The brackets are a superset of dot notation
- Every trace of ix should be removed and replaced with loc or iloc

In [0]:
data['CITY'] # instead of data.CITY

In [0]:
data.loc[:, ['CITY', 'STABBR']] # instead of data.ix

In [0]:
data.iloc[:, 1:3]

**About NA**

- only use **isna** and **notna**,matching the names of the other missing value methods **dropna** and **fillna**.
- You can also avoid ever using **notna** since Pandas provides the inversion operator, ~ to invert boolean DataFrames.

In [0]:
data['UNITID'].isna() # instead of: data['UNITID'].isnull()

**Use the Pandas method over any built-in Python function with the same name.**
- sum(), max(), min()

- Calling sum() essentially creates a Python for loop to iterate through each value one at a time. On the other hand, calling.sum() executes the internal Pandas sum method which is written in C and much faster than iterating with a Python for loop.

In [22]:

data['PBI'].dropna().sum() # instead of: sum(data['PBI'].dropna())

101.0

**Standardizing groupby Aggregation**
- df.groupby('grouping column').agg({'aggregating column': 'aggregating function'})
- df.groupby(['gropuing column']).agg({'colum1': ['sum', 'max'],'column2':['min']})
- Avoid using a MultiIndex. Flatten it after a call to groupby by renaming columns and resetting the index.

In [28]:
data.groupby('CITY').agg({'PBI':['max', 'min']}).head() # instead of: data.groupby('CITY').['PBI'].aggregate('max')

PBI     
          max  min
CITY              
ARTESIA   0.0  0.0
Aberdeen  0.0  0.0
Abilene   0.0  0.0
Abingdon  0.0  0.0
Abington  0.0  0.0

In [32]:
# flatten

df = data.groupby('CITY').agg({'PBI':['max', 'min']}).head()
df.columns = ['PBI_MAX', 'PBI_MIN'] # manually assign names
df.reset_index()

,CITY,PBI_MAX,PBI_MIN
0,ARTESIA,0.0,0.0
1,Aberdeen,0.0,0.0
2,Abilene,0.0,0.0
3,Abingdon,0.0,0.0
4,Abington,0.0,0.0


**Use pivot_table when comparing groups**
- use pivot tables to compare values across groups and a groupby when want to continue an analysis. 
- Pivot tables as a finished product. The result of a groupby is going to be in tidy form, which lends itself to easier subsequent analysis, but isn’t as interpretable.
- Use pivot_table over unstack or pivot

In [33]:
data.groupby(['STABBR','CITY']).agg({'PBI':['max', 'min']}).head()

PBI     
                  max  min
STABBR CITY               
AK     Anchorage  0.0  0.0
       Barrow     0.0  0.0
       Fairbanks  0.0  0.0
       Juneau     0.0  0.0
       Palmer     0.0  0.0

In [36]:
data.pivot_table(index = ['STABBR', 'CITY'], values = 'PBI', aggfunc=['max','min']).head()

max  min
                  PBI  PBI
STABBR CITY               
AK     Anchorage  0.0  0.0
       Barrow     0.0  0.0
       Fairbanks  0.0  0.0
       Juneau     0.0  0.0
       Palmer     0.0  0.0

**Use melt over stack because it allows you to rename columns and it avoids a MultiIndex**

In [41]:
data = data.loc[:, ['OPEID', 'CITY', 'HCM2', 'PBI']]
data.head(2)

,OPEID,CITY,HCM2,PBI
0,100200,Normal,0,0.0
1,105200,Birmingham,0,0.0


In [47]:
data.melt(id_vars = ['CITY','OPEID'], var_name = 'measures', value_name = 'numbers' ).head() # could rename the column name

,CITY,OPEID,measures,numbers
0,Normal,100200,HCM2,0.0
1,Birmingham,105200,HCM2,0.0
2,Montgomery,2503400,HCM2,0.0
3,Huntsville,105500,HCM2,0.0
4,Montgomery,100500,HCM2,0.0


In [48]:
data.stack()

0     OPEID              100200
      CITY               Normal
      HCM2                    0
      PBI                     0
1     OPEID              105200
      CITY           Birmingham
      HCM2                    0
      PBI                     0
2     OPEID             2503400
      CITY           Montgomery
      HCM2                    0
      PBI                     0
3     OPEID              105500
      CITY           Huntsville
      HCM2                    0
      PBI                     0
4     OPEID              100500
      CITY           Montgomery
      HCM2                    0
      PBI                     0
5     OPEID              105100
      CITY           Tuscaloosa
      HCM2                    0
      PBI                     0
6     OPEID              100700
      CITY       Alexander City
      HCM2                    0
      PBI                     0
7     OPEID              100800
      CITY               Athens
                     ...       
7102  OP